In [ ]:
import Merge_Cartesian as MC
import numpy as np
from decimal import *
import math
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
%matplotlib widget

In [ ]:
def float_to_decimal(f):
    return Decimal(str(f))

arr_float_to_decimal = np.vectorize(float_to_decimal)

In [ ]:
def arrs_lcm_gcd(starting_number,finishing_number,increment):

    number_of_numbers = math.ceil((float_to_decimal(finishing_number)-float_to_decimal(starting_number))/float_to_decimal(increment))+1

    x = np.linspace(starting_number,finishing_number,number_of_numbers)
    y = np.linspace(starting_number,finishing_number,number_of_numbers)

    x_grid, y_grid = arr_float_to_decimal(np.meshgrid(x, y))
    return MC.get_lcm_gcd(x_grid,y_grid)

In [ ]:
def plot_lcm(starting_number,finishing_number,increment,ax ):
    lcm,gcd = arrs_lcm_gcd(starting_number,finishing_number,increment)
    MC.plot_fanout_colour(lcm,ax,"LCM of numbers " + str(starting_number) + " to" + str(finishing_number)+ " in increments of "+str(increment),False)
    
def plot_gcd(starting_number,finishing_number,increment,ax ):
    lcm,gcd = arrs_lcm_gcd(starting_number,finishing_number,increment)
    MC.plot_fanout_colour(gcd,ax,"GCD of numbers " + str(starting_number) + " to" + str(finishing_number)+ " in increments of "+str(increment),False)
    
def plot_lcm_gcd(starting_number,finishing_number,increment,ax ):
    ax[0].clear()
    lcm,gcd = arrs_lcm_gcd(starting_number,finishing_number,increment)
    MC.plot_fanout_colour(lcm,ax[0],"LCM of numbers " + str(starting_number) + " to" + str(finishing_number)+ " in increments of "+str(increment),False)
    MC.plot_fanout_colour(gcd,ax[1],"GCD of numbers " + str(starting_number) + " to" + str(finishing_number)+ " in increments of "+str(increment),False)
    
def add_float_as_decimal(a,b):
    a_d = Decimal(str(a))
    b_d = Decimal(str(b))
    
    return float(a_d + b_d)

In [ ]:
fig_c, ax_c = plt.subplots(1,2)        
# Intialize        
# starting_number = 1
# finishing_number = 100
# increment = 1

colour_bar = plot_lcm_gcd(1,100,1,ax_c)

increment_button = widgets.Button(description = "step forward", layout=widgets.Layout(width='auto'))
decrement_button = widgets.Button(description = "step backward", layout=widgets.Layout(width='auto'))
increment_text = widgets.FloatText(description = 'val', value=0.1)

start_input = widgets.FloatText(description = 'Start Num', value=1)
end_input = widgets.FloatText(description = 'End Num', value=100)
inc_input = widgets.FloatText(description = '++ Num', value=1)
plot_button = widgets.Button(description = "Plot")

output = widgets.Output()


def on_increment_click(b):
    inc_input.value = add_float_as_decimal(increment_text.value,inc_input.value)
    plot_lcm_gcd(start_input.value,end_input.value,inc_input.value,ax_c)
 
def on_decrement_click(b):
    inc_input.value = add_float_as_decimal(-increment_text.value,inc_input.value)
    plot_lcm_gcd(start_input.value,end_input.value,inc_input.value,ax_c)
            
def handle_input_change(c):
    plot_lcm_gcd(start_input.value,end_input.value,inc_input.value,ax_c)
            

increment_button.on_click(on_increment_click)
decrement_button.on_click(on_decrement_click)
plot_button.on_click(handle_input_change)

increment_grid = widgets.GridspecLayout(1,3)
increment_grid[0,0] = decrement_button
increment_grid[0,1] = increment_button
increment_grid[0,2] = increment_text

start_stop = widgets.HBox([start_input,end_input,inc_input,plot_button])

display(increment_grid,start_stop)